In [1]:
from actions import ActionsWorkflow, ActionsJobs, ActionsArtifacts
import LogExtractor as extractor

from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer, ListFlowable, ListItem, Spacer, Image
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tempfile
import os
import kaleido


## Retrieving test data and processing 

In [2]:
repo_path = 'MagaluCloud/s3-specs'
query_size = 10


In [3]:

workflow = ActionsWorkflow(repository=repo_path, query_size=query_size)
jobs = ActionsJobs(repo_path, workflow)

all_workflows_jobs = pd.DataFrame()

for id in workflow.df['databaseId']:
    tmp = jobs.get_jobs(id)
    all_workflows_jobs = pd.concat([all_workflows_jobs, tmp])

DataFrame successfully saved to ./bin/actionsWorflow.parquet


In [4]:
artifacts = ActionsArtifacts(repository=repo_path)
all_tests_df = pd.DataFrame()
all_times_df = pd.DataFrame()
all_failures_df = pd.DataFrame()

for path in artifacts.paths:
    artifact = extractor.PytestArtifactLogExtractor(path)
    pytest_tests_status	, pytest_run_times, pytest_failures_errors = artifact.log_to_df()
    all_tests_df = pd.concat([all_tests_df, pytest_tests_status])
    all_times_df = pd.concat([all_times_df, pytest_run_times])
    all_failures_df = pd.concat([all_failures_df, pytest_failures_errors])

In [5]:
all_tests_df.groupby('status').value_counts()

status  category                arguments                                                                                                     databaseId 
FAILED  versioning_cli_test.py  mgc object-storage buckets delete {bucket_name} --no-confirm --recursive --raw-the bucket may not be empty]   13269149128    1
                                mgc object-storage objects delete {bucket_name}/{object_key} --no-confirm-]                                   13269149128    1
PASSED  acl_test.py             authenticated-read]                                                                                           0              4
                                private]                                                                                                      0              4
                                public-read-write]                                                                                            0              4
                                                   

In [6]:
# Trocar databaseId por jobId
display(all_tests_df)
display(all_times_df[all_times_df['databaseId'].notna()])
display(all_failures_df)

,status,category,arguments,databaseId
pytest_tests_status,,,,
test_boto_upload_object_with_cold_storage_class,PASSED,cold_storage_test.py,None,0
test_boto_object_with_custom_metadata_acls_and_storage_class,PASSED,cold_storage_test.py,None,0
test_boto_list_objects_with_cold_storage_class,PASSED,cold_storage_test.py,None,0
test_boto_change_object_class_to_cold_storage,PASSED,cold_storage_test.py,None,0
test_boto_multipart_upload_with_cold_storage_class,PASSED,cold_storage_test.py,None,0
...,...,...,...,...
test_delete_bucket_with_objects_with_versions,PASSED,versioning_cli_test.py,mgc object-storage buckets delete {bucket_name...,13269265723
test_delete_bucket_with_objects_with_versions,PASSED,versioning_cli_test.py,rclone rmdir {profile_name}:{bucket_name}-Buck...,13269265723
test_delete_bucket_with_objects_with_versions,PASSED,versioning_cli_test.py,aws --profile {profile_name} s3 rb s3://{bucke...,13269265723


,total,num,avg,min,durationType,databaseId
pytest_run_times,,,,,,
test_boto_multipart_upload_with_cold_storage_class,2.507,1,2.507,2.507,teardown duration,0
test_boto_object_with_custom_metadata_acls_and_storage_class,2.202,1,2.202,2.202,teardown duration,0
test_boto_upload_object_with_cold_storage_class,2.143,1,2.143,2.143,teardown duration,0
test_boto_change_object_class_to_cold_storage,1.331,1,1.331,1.331,teardown duration,0
test_boto_list_objects_with_cold_storage_class,1.121,1,1.121,1.121,teardown duration,0
...,...,...,...,...,...,...
test_delete_bucket_with_objects_with_versions,5.903,4,1.705,0.618,call duration,13269265723
versioned_bucket_with_one_object,46.476,8,5.772,5.330,fixture duration,13269265723
s3_client,1.409,8,0.074,0.067,fixture duration,13269265723


,status,category,error,error_details,databaseId
pytest_failures_errors,,,,,
test_delete_object_with_versions,FAILED,versioning_cli_test.py,AssertionError,Command failed with error: Error: (NoSuchBuck...,13269149128
test_delete_bucket_with_objects_with_versions,FAILED,versioning_cli_test.py,assert,'the bucket may not be empty' in 'WARN\tgithub...,13269149128


## Plotter

In [7]:
# Setup
error_distribution_df = all_tests_df[['category', 'status']]
error_passed_info = error_distribution_df.groupby(['status','category']).value_counts()

status_freq_df = pd.concat([error_passed_info.FAILED, error_passed_info.PASSED], axis=1).fillna(0).astype(int)
status_freq_df.columns = ['FAILED','PASSED']
status_freq_df = status_freq_df.reset_index()
error_df = all_failures_df
display(error_distribution_df)
display(status_freq_df)

,category,status
pytest_tests_status,,
test_boto_upload_object_with_cold_storage_class,cold_storage_test.py,PASSED
test_boto_object_with_custom_metadata_acls_and_storage_class,cold_storage_test.py,PASSED
test_boto_list_objects_with_cold_storage_class,cold_storage_test.py,PASSED
test_boto_change_object_class_to_cold_storage,cold_storage_test.py,PASSED
test_boto_multipart_upload_with_cold_storage_class,cold_storage_test.py,PASSED
...,...,...
test_delete_bucket_with_objects_with_versions,versioning_cli_test.py,PASSED
test_delete_bucket_with_objects_with_versions,versioning_cli_test.py,PASSED
test_delete_bucket_with_objects_with_versions,versioning_cli_test.py,PASSED


,category,FAILED,PASSED
0,versioning_cli_test.py,2,22
1,acl_test.py,0,232
2,cold_storage_test.py,0,20
3,list-buckets_test.py,0,16
4,locking_test.py,0,4
5,policies_test.py,0,84
6,presigned-urls_test.py,0,12
7,profiles_policies_test.py,0,24
8,unique-bucket-name_test.py,0,8
9,versioning_test.py,0,8


In [8]:
def error_distribution_pie_chat_plt(error_df):

    plt.figure(figsize=(5, 5))
    plt.pie(
        error_passed_info['FAILED'].values,
        labels=error_passed_info['FAILED'].index.to_list(),
        autopct='%1.1f%%',
        startangle=90,
        colors=['firebrick', 'lightgreen', 'lightskyblue'],
    )
    plt.legend(title="Error Types", loc="upper right", bbox_to_anchor=(1.3, 1))
    plt.title('Failure Distribution')
    plt.tight_layout()

    #return fig

In [9]:
def error_distribution_pie_chart(error_df):
    # Filter for FAILED status
    failed_df = error_df[error_df['status'] == 'FAILED']

    # Group by category and count the number of FAILED statuses
    failed_counts = failed_df.groupby('category').size().reset_index(name='count')

    # Create the pie chart
    fig = px.pie(
        failed_counts, 
        names="category",  # Use 'category' for pie slice labels
        values="count",    # Use 'count' for pie slice sizes
        title="Distribuição de falhas por categoria",
        color_discrete_sequence=px.colors.sequential.RdBu,
    )

    fig.update_layout(
    #   width=400,  # Set the width of the plot (in pixels)
    #  height=400,  # Set the height of the plot (in pixels)
        margin=dict(l=20, r=20, t=40, b=20)  # Adjust margins if needed
    )

    # Make the pie chart circle bigger by adjusting the marker size
    fig.update_traces(
        marker=dict(line=dict(color='white', width=2)),  # Optional: Add a white border
        textposition='inside',  # Display text inside the slices
        textinfo='percent+label'  # Show percentage and label
    )

    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        fig.write_image(tmpfile.name, format="png", width=800, height=400)
        return tmpfile.name
    

In [10]:
def plot_category_errors_bar(error_df):
    # Calculate the frequency of errors per category
    error_freq_df = error_df.groupby(['category', 'error']).size().reset_index(name='frequency')

    # Create the bar plot
    fig = px.bar(
        error_freq_df, 
        x="category", 
        y="frequency", 
        color="error",  # Use a discrete color sequence
        color_discrete_sequence=px.colors.sequential.RdBu,
        title="Frequência de erros por categoria",
        labels={'frequency': 'Frequency of Errors', 'category': 'Category'},
    )

    # Adjust layout to control bar width
    fig.update_layout(
        xaxis_title="Category",
        yaxis_title="Frequency of Errors",
        barmode='stack',  
        bargroupgap=0.1,  
        width=600,
        margin=dict(l=20, r=20, t=40, b=20)  
    )

    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        fig.write_image(tmpfile.name, format="png", width=800, height=400)
        return tmpfile.name

In [11]:
def categories_failures_passed_rate(status_freq_df):
    # Calcular totais e percentuais
    status_freq_df['Total'] = status_freq_df['PASSED'] + status_freq_df['FAILED']
    status_freq_df['PASSED_pct'] = (status_freq_df['PASSED'] / status_freq_df['Total']) * 100
    status_freq_df['FAILED_pct'] = (status_freq_df['FAILED'] / status_freq_df['Total']) * 100

    # Transformar dados para formato longo
    status_freq_long = status_freq_df.melt(
        id_vars=['category'], 
        value_vars=['PASSED_pct', 'FAILED_pct'], 
        var_name='Status', 
        value_name='Percentage'
    )

    # Criar coluna com valores reais correspondentes
    status_freq_long['Real Value'] = status_freq_long.apply(
        lambda row: status_freq_df.loc[status_freq_df['category'] == row['category'], row['Status'].replace('_pct', '')].values[0], 
        axis=1
    )

    # Criar gráfico
    fig = px.bar(
        status_freq_long, 
        x="category", 
        y="Percentage", 
        color="Status", 
        barmode='stack', 
        title="Proporção de testes Aprovados/Falho",
        labels={'Percentage': 'Percentage'},
        text=status_freq_long["Real Value"]  # Exibir valores reais nas barras
    )

    # Ajustar layout para exibir os valores dentro das barras
    fig.update_traces(texttemplate='%{text}', textposition='inside')
    fig.update_yaxes(title='Percentage')
    fig.update_xaxes(title='Category')

    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        fig.write_image(tmpfile.name, format="png", width=800, height=400)
        return tmpfile.name

In [12]:
def build_all_graphs():
    return {
        'category_errors_bar': plot_category_errors_bar(error_df),
        'error_distribution_pie': error_distribution_pie_chart(error_distribution_df),
        'failures_passed_rate': categories_failures_passed_rate(status_freq_df),
    }

build_all_graphs()

{'category_errors_bar': '/tmp/tmp1xe00lzm.png',
 'error_distribution_pie': '/tmp/tmpfu31gf3y.png',
 'failures_passed_rate': '/tmp/tmpqog7xzkr.png'}

## Creating pdf

In [13]:
p = all_tests_df.groupby(['category','status']).size().unstack('status').fillna(0)
p['total'] = p['FAILED'] + p['PASSED']
p

status,FAILED,PASSED,total
category,,,
acl_test.py,0.0,232.0,232.0
cold_storage_test.py,0.0,20.0,20.0
list-buckets_test.py,0.0,16.0,16.0
locking_test.py,0.0,4.0,4.0
policies_test.py,0.0,84.0,84.0
presigned-urls_test.py,0.0,12.0,12.0
profiles_policies_test.py,0.0,24.0,24.0
unique-bucket-name_test.py,0.0,8.0,8.0
versioning_cli_test.py,2.0,22.0,24.0


In [14]:
# Generate a tuple with the category name and the summed up values of all index of said class
def get_time(metric):
    return pd.Series(dict(map(lambda t, x: (x, all_times_df.loc[all_times_df.index == t, metric].sum()), all_tests_df.index.unique(), all_tests_df.category.unique())))

total_times = get_time('total')
avg_time_test = get_time('avg')
min_test_time  = get_time('min')

cpf_df = all_tests_df.groupby(['category','status']).size().unstack('status').fillna(0).astype(int)
cpf_df['total'] = cpf_df.sum(axis=1)
time_count_df = pd.concat([cpf_df['PASSED'], cpf_df['FAILED'], cpf_df['total'], min_test_time, avg_time_test, total_times], axis=1)
time_count_df.columns = ['num_passed', 'num_failed', 'total_runs', 'min_test_time', 'avg_test_time', 'total_duration']

report_df = pd.DataFrame()
report_df['name'] = all_tests_df['category'].unique()
report_df = report_df.set_index('name')

report_df = pd.concat([report_df, time_count_df], axis=1)
report_df

,num_passed,num_failed,total_runs,min_test_time,avg_test_time,total_duration
cold_storage_test.py,20,0,20,19.314,19.314,19.314
presigned-urls_test.py,12,0,12,19.223,19.223,19.223
list-buckets_test.py,16,0,16,18.272,18.272,18.272
unique-bucket-name_test.py,8,0,8,18.168,18.168,18.168
acl_test.py,232,0,232,33.804,33.804,33.804
versioning_cli_test.py,22,2,24,18.151,18.151,18.151
versioning_test.py,8,0,8,18.201,18.201,18.201
policies_test.py,84,0,84,17.338,17.338,17.338
locking_test.py,4,0,4,54.964,54.964,54.964
profiles_policies_test.py,24,0,24,50.807,54.577,164.509


In [15]:
# Generate tuple with error values 
errors_df = all_failures_df
errors_df

,status,category,error,error_details,databaseId
pytest_failures_errors,,,,,
test_delete_object_with_versions,FAILED,versioning_cli_test.py,AssertionError,Command failed with error: Error: (NoSuchBuck...,13269149128
test_delete_bucket_with_objects_with_versions,FAILED,versioning_cli_test.py,assert,'the bucket may not be empty' in 'WARN\tgithub...,13269149128


Blocos que vão existir

Topo: Contendo Informações básicas do relatório e se possível alguns campos em branco e também a data de quando o código foi executado

Texto: Informações gerais do número de acertos e erros

Tabela: Contem o dataframe report_df, mas estilizado

Gráficos: fica pra dps

In [16]:
df = report_df.reset_index().round(2)
# Function to create PDF
def create_pdf(df, df_errors):
    # A4 size dimensions
    width, height = A4

    # Set 10% margin
    margin = 0.1 * width

    # Create PDF with margins
    doc = SimpleDocTemplate("report_v0.pdf", pagesize=A4,
                            leftMargin=margin, rightMargin=margin, topMargin=0.1*height, bottomMargin=0.1*height)

    # Styles
    styles = getSampleStyleSheet()
    heading_style = styles['Heading1']
    normal_style = styles['Normal']
    normal_style.alignment = 0  # 0 for left alignment

    bold_style = ParagraphStyle(
        name="Bold",
        parent=normal_style,
        fontName="Helvetica-Bold",
        fontSize=12
    )

    # Create the story (content) for the PDF
    story = []

    # Add title with fields
    story.extend(create_title(heading_style,normal_style))

    # Add each section to the story
    story.extend(create_execution_summary(df, normal_style, bold_style))
    story.extend(create_detailed_results(df, normal_style, bold_style, width, margin))
    story.extend(create_errors_summary(df_errors, normal_style, bold_style, width, margin))
    story.extend(create_graphs(normal_style, bold_style, width, margin))
    # Build PDF
    doc.build(story)

# Function to create the title zone and text on the pdf
def create_title(heading_style, normal_style):
    # Initialize the story list
    story = []

    # Get current date and time
    agora = datetime.now()
    horario_dia = agora.strftime("%d/%m/%Y %H:%M:%S")

    # Create the title
    title_text = "Sumário de Resultados dos Testes"
    title_paragraph = Paragraph(f"<b>{title_text}</b>", heading_style)

    # Create the formatted text for the execution date on the right side
    right_date_style = ParagraphStyle(
        "RightDateStyle", parent=normal_style, alignment=2, fontSize=10
    )
    date_paragraph = Paragraph(horario_dia, right_date_style)

    # Add title and date to the story as separate elements
    story.append(title_paragraph)
    story.append(date_paragraph)

    # Create the formatted text for the execution date, system version, and environment
    execution_paragraph = Paragraph(f"Data da Execução: ", normal_style)
    version_paragraph = Paragraph("Versão do Sistema: ", normal_style)
    environment_paragraph = Paragraph("Ambiente: ", normal_style)

    # Add other paragraphs to the story
    story.append(execution_paragraph)
    story.append(Spacer(1, 12))  # Spacer between execution and version
    story.append(version_paragraph)
    story.append(Spacer(1, 12))  # Spacer between version and environment
    story.append(environment_paragraph)

    story.append(Spacer(1, 24))  # Add space at the end

    # Return the complete story
    return story

# Function to create execution summary table with bullet points
def create_execution_summary(df, normal_style, bold_style):
    story = []
    story.append(Paragraph("Resumo Geral", bold_style))
    story.append(Spacer(1, 6))

    # Criando a lista de resumo corretamente
    summary_data = {
        'Total de Testes:': df['total_runs'].sum(),
        'Testes Bem-Sucedidos:': df['num_passed'].sum(),
        'Testes com Falha:': df['num_failed'].sum(),
        #'Taxa de Sucessos/Falha:': f"{(df['num_failed'] / df['num_passed'] * 100).round(2)}%",  # Round to 2 decimal places
        'Tempo Mínimo de Execução:': f"{df['min_test_time'].min():.2f} s",
        'Tempo Médio de Execução:': f"{df['avg_test_time'].mean():.2f} s",
        'Duração Total dos Testes:': f"{df['total_duration'].sum():.2f} s"
    }

    # Criando a lista com bullet points
    bullet_points = [
        ListItem(Paragraph(f"<b>{key}</b> {value}", normal_style), leftIndent=20, spaceAfter=6)
        for key, value in summary_data.items()
    ]

    # Criando o ListFlowable
    list_flowable = ListFlowable(bullet_points, bulletType='bullet', leftIndent=20)

    # Adicionando ao relatório
    story.append(list_flowable)
    story.append(Spacer(1, 24))

    return story

# Function to create detailed results table
def create_detailed_results(df, normal_style, bold_style, width, margin):
    story = []
    story.append(Paragraph("Detalhamento dos Testes", bold_style))
    story.append(Spacer(1, 12))
    df_renamed = df.copy()  # Create a copy of the DataFrame
    df_renamed.columns = [
        'Categoria de Teste', 
        'Testes Bem-Sucedidos', 
        'Falhas', 
        'Execuções', 
        'Tempo Mínimo de Execução', 
        'Tempo Médio', 
        'Duração Total'
    ]


    # Dropping the specified columns
    df_renamed = df_renamed.drop(columns=['Tempo Mínimo de Execução'])

    df_renamed['Tempo Médio'] = df_renamed['Tempo Médio'].astype(str) + ' sec'
    df_renamed['Duração Total'] = df_renamed['Duração Total'].astype(str) + ' sec'

    # Prepare the detailed data for the table
    detailed_tests_data = [[Paragraph(str(value), normal_style) for value in df_renamed.columns.tolist()]]  # Add header
    detailed_tests_data.extend(
        [[Paragraph(str(value), normal_style) for value in row] for row in df_renamed.values.tolist()]
    )

    # Calculate available width after applying margins
    available_width = width - 2 * margin  # Subtracting left and right margins

    # Define column proportions
    proportions = [0.3, 0.15, 0.15, 0.15, 0.2]  # Example proportions

    total_proportion = sum(proportions)
    if total_proportion > 1:
        proportions = [p / total_proportion for p in proportions]  # Scale proportions to fit within 1

    # Calculate column widths based on the available width
    col_widths = [available_width * p for p in proportions]

    # Create the table
    detailed_table = Table(detailed_tests_data, colWidths=col_widths)
    detailed_table.setStyle(TableStyle([('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                                        ('GRID', (0, 0), (-1, -1), 0.5, colors.black)]))
    story.append(detailed_table)
    story.append(Spacer(1, 24))

    return story

def create_errors_summary(df, normal_style, bold_style, width, margin):
    """
    Creates a summary of errors in a PDF document.

    :param df: DataFrame containing error data.
    :param normal_style: Style for normal text.
    :param bold_style: Style for bold text.
    :param width: Width of the page.
    :param margin: Margin size.
    :return: A list of elements to be added to the PDF.
    """
    story = []
    story.append(Paragraph("Resumo dos Erros", bold_style))
    story.append(Spacer(1, 12))

    # Create a copy of the DataFrame and reset the index
    df_copy = df.copy().reset_index()
    df_copy.columns = [
        'Nome',
        'Status',
        'Categoria do Teste',
        'Tipo de erro',
        'Detalhes do erro (100 caracteres)',
        'JobId',
    ]

    df_copy = df_copy.drop('Detalhes do erro (100 caracteres)', axis=1)
    display(df_copy)


    # Prepare the detailed data for the table
    detailed_tests_data = [[Paragraph(str(value), normal_style) for value in df_copy.columns.tolist()]]  # Add header
    detailed_tests_data.extend(
        [[Paragraph(str(value), normal_style) for value in row] for row in df_copy.values.tolist()]
    )

    # Calculate available width after applying margins
    available_width = width - 2 * margin  # Subtracting left and right margins

    # Define column proportions
    proportions = [0.3, 0.15, 0.15, 0.15, 0.2, 0.1]  # Example proportions

    # Calculate column widths based on the available width
    col_widths = [available_width * p for p in proportions]

    # Create the table
    detailed_table = Table(detailed_tests_data, colWidths=col_widths)
    detailed_table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center-align all cells
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),  # Add grid lines
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Bold header font
    ]))
    story.append(detailed_table)
    story.append(Spacer(1, 24))

    return story

def create_graphs(normal_style, bold_style, width, margin):
    graph_files = build_all_graphs()
    story = []

    # Add a title to the PDF
    story.append(Spacer(1, 12))

    story.append(Paragraph("Visualização de dados", bold_style))
    story.append(Spacer(1, 12))

    # Add the bar chart to the PDF
    story.append(Image(graph_files['category_errors_bar'], width=500, height=250))  # Adjust size as needed
    story.append(Spacer(1, 24))

    # Add the pie chart to the PDF
    story.append(Image(graph_files['error_distribution_pie'], width=500, height=250))  # Adjust size as needed
    story.append(Spacer(1, 24))

    # Add the pass/fail rate bar chart to the PDF
    story.append(Image(graph_files['failures_passed_rate'], width=500, height=250))  # Adjust size as needed
    story.append(Spacer(1, 24))

    return story

# Gerar o PDF
create_pdf(df, errors_df)


,Nome,Status,Categoria do Teste,Tipo de erro,JobId
0,test_delete_object_with_versions,FAILED,versioning_cli_test.py,AssertionError,13269149128
1,test_delete_bucket_with_objects_with_versions,FAILED,versioning_cli_test.py,assert,13269149128


## Plotting

### Error distribution (Pie chart)

In [17]:
error_distribution_df = all_tests_df[['category', 'status']]
error_passed_info = error_distribution_df.groupby(['status','category']).value_counts()

status_freq_df = pd.concat([error_passed_info.FAILED, error_passed_info.PASSED], axis=1).fillna(0).astype(int)
status_freq_df.columns = ['FAILED','PASSED']
status_freq_df = status_freq_df.reset_index()


In [18]:
def error_distribution_pie_chat(error_df):
    # Filter for FAILED status
    failed_df = error_df[error_df['status'] == 'FAILED']

    # Group by category and count the number of FAILED statuses
    failed_counts = failed_df.groupby('category').size().reset_index(name='count')

    # Create the pie chart
    fig = px.pie(
        failed_counts, 
        names="category",  # Use 'category' for pie slice labels
        values="count",    # Use 'count' for pie slice sizes
        title="Distribution of FAILED Tests by Category",
        color_discrete_sequence=px.colors.sequential.RdBu,
    )

    fig.update_layout(
    #   width=400,  # Set the width of the plot (in pixels)
    #  height=400,  # Set the height of the plot (in pixels)
        margin=dict(l=20, r=20, t=40, b=20)  # Adjust margins if needed
    )

    # Make the pie chart circle bigger by adjusting the marker size
    fig.update_traces(
        marker=dict(line=dict(color='white', width=2)),  # Optional: Add a white border
        textposition='inside',  # Display text inside the slices
        textinfo='percent+label'  # Show percentage and label
    )

    # Show the pie chart
    fig.show()

    

In [19]:
def error_distribution_pie_chat_plt(error_df):

    plt.figure(figsize=(5, 5))
    plt.pie(
        error_passed_info['FAILED'].values,
        labels=error_passed_info['FAILED'].index.to_list(),
        autopct='%1.1f%%',
        startangle=90,
        colors=['firebrick', 'lightgreen', 'lightskyblue'],
    )
    plt.legend(title="Error Types", loc="upper right", bbox_to_anchor=(1.3, 1))
    plt.title('Failure Distribution')
    plt.tight_layout()
    plt.show()



### Proportion of Passed/Failed (%) with Real Values Displayed

In [20]:

# Calcular totais e percentuais
status_freq_df['Total'] = status_freq_df['PASSED'] + status_freq_df['FAILED']
status_freq_df['PASSED_pct'] = (status_freq_df['PASSED'] / status_freq_df['Total']) * 100
status_freq_df['FAILED_pct'] = (status_freq_df['FAILED'] / status_freq_df['Total']) * 100

# Transformar dados para formato longo
status_freq_long = status_freq_df.melt(
    id_vars=['category'], 
    value_vars=['PASSED_pct', 'FAILED_pct'], 
    var_name='Status', 
    value_name='Percentage'
)

# Criar coluna com valores reais correspondentes
status_freq_long['Real Value'] = status_freq_long.apply(
    lambda row: status_freq_df.loc[status_freq_df['category'] == row['category'], row['Status'].replace('_pct', '')].values[0], 
    axis=1
)

# Criar gráfico
fig = px.bar(
    status_freq_long, 
    x="category", 
    y="Percentage", 
    color="Status", 
    barmode='stack', 
    title="Proportion of Passed/Failed (%) with Real Values Displayed",
    labels={'Percentage': 'Percentage'},
    text=status_freq_long["Real Value"]  # Exibir valores reais nas barras
)

# Ajustar layout para exibir os valores dentro das barras
fig.update_traces(texttemplate='%{text}', textposition='inside')
fig.update_yaxes(title='Percentage')
fig.update_xaxes(title='Category')

fig.show()


### Error type rate


In [21]:
def plot_category_errors_bar(df):
    # Calculate the frequency of errors per category
    error_freq_df = df.groupby(['category', 'error']).size().reset_index(name='frequency')

    # Create the bar plot
    fig = px.bar(
        error_freq_df, 
        x="category", 
        y="frequency", 
        color="error",  # Use a discrete color sequence
        color_discrete_sequence=px.colors.sequential.RdBu,
        title="Frequency of Errors by Category",
        labels={'frequency': 'Frequency of Errors', 'category': 'Category'},
    )

    # Adjust layout to control bar width
    fig.update_layout(
        xaxis_title="Category",
        yaxis_title="Frequency of Errors",
        barmode='stack',  
        bargroupgap=0.1,  
        width=600,
        margin=dict(l=20, r=20, t=40, b=20)  
    )

    # Show the plot
    fig.show()